In [1]:
import torch
from torchvision import transforms
from dataset import *

In [1]:
data_loader = IrisDataModule(
    data_dir= [
        "../Datasets/train_iris_nd_crosssensor_2013", 
        "../Datasets/train_iris_casia_v4",
        "../Datasets/train_iris_nd_0405",
        "../Datasets/train_iris_utris_v1"
    ],
    predic_data_dir="../Datasets/iris_verification_NDCSI2013_01_05",
    train_transforms=transforms.Compose([
        transforms.Resize((112,112)),
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.RandomInvert(p=0.5),
        transforms.Normalize(mean=[0.485], std=[0.229]),
        transforms.RandomAdjustSharpness(sharpness_factor=2,p=0.5),
        transforms.RandomAutocontrast(p=0.5),
        transforms.RandomAffine(
            degrees=10,
            translate=(0.1, 0.1),
            scale=(0.8, 1.3),
            shear=20,
            fill=0,
        ),
        transforms.RandomErasing(
            p=0.5, 
            scale=(0.02, 0.33), 
            ratio=(0.3, 3.3), 
            value=0, 
            inplace=False
        ),
    ]), 
    predict_transforms=transforms.Compose([
        transforms.Resize((112,112)),
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=[0.485], std=[0.229]),
    ])
)

NameError: name 'IrisDataModule' is not defined

In [ ]:
data_loader.setup()
for batch in data_loader.train_dataloader():
    import matplotlib.pyplot as plt
    fig, axes = plt.subplots(3,3)
    for i, axs in enumerate(axes):
        for j, ax in enumerate(axs):
            ax.imshow(batch[0][i*3+j][0,:,:])
            ax.set_title(str(batch[1][i*3+j]))
    fig.show()
    break